<font size="6">Машинный перевод</font>

# Задача машинного перевода

Ранее мы рассматривали в основном задачу классификации: каждому тексту сопоставляется метка класса из заданного списка.

Другая важная задача обработки естественного языка — машинный перевод: последовательности на одном языке сопоставляется последовательность на другом языке, передающая тот же смысл.

Рассмотрим задачу перевода с английского на французский язык:
- Исходное предложение: I am a student.
- Целевое предложение: Je suis étudiant.

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-content/L04/out/machine_translation.png" width="600"></center>

Что нужно для перевода текста?
1. Понимать отдельные слова
2. Понимать взаимодействие между словами (синтаксис)
3. Переводить слова (с учетом контекста)
4. Составлять осмысленный и связный текст

Для перевода текста можно разбить исходное предложение на несколько фрагментов, затем переводить его по фразам.

Однако у такого подхода есть проблемы:
- одному слову в исходном предложении (*forced*) может соответствовать несколько слов в целевом предложении (*a forcé*)
- порядок слов может меняться (*exceptional measures* vs. *des mesures exceptionnelles*)

Следовательно, возникают сложности с построением выравнивания слов (word alignment).

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-content/L04/out/word_alignment.png" width="800"></center>

Другая возможность — прочитать все исходное предложение целиком, проанализировать его значение, а затем произвести перевод.

В этом случае модель состоит из двух блоков:
- кодировщик: строит векторное представление для исходного предложения (кодирует);
- декодировщик: генерирует целевое предложение на основе этого векторного представления (раскодирует).

<center><img src="https://i.ibb.co/h2srmnP/encoder-decoder.png" width="800"></center>

<center><em>Source: <a href="https://blog.paperspace.com/introduction-to-neural-machine-translation-with-bahdanaus-attention/">Introduction to Neural Machine Translation</a></em></center>

В качестве блоков кодировщика и декодировщика могут использоваться нейронные сети с различной архитектурой.
1. Изначально это были рекуррентные нейронные сети, которые хорошо подходят для обработки последовательных данных;
2. Позже была предложена архитектура Трансформер, которая лежит в основе современных языковых моделей.

В сегодняшней лекции мы рассмотрим обе эти архитектуры и их применение в задаче машинного перевода.

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L04/rnn_transformer.png" width="700"></center>

<center><em>Source: <a href="https://shchegrikovich.substack.com/p/rnn-vs-transformers-or-how-scalability">RNN vs Transformers</a></em></center>

# Рекуррентные нейронные сети

**Рекуррентные нейронные сети (recurrent neural networks, RNN)** — это класс нейронных сетей, которые применяются для обработки последовательных данных.

Они применяются в широком перечне задач: от **распознавания речи** до **генерации подписей** к изображениям.

Эти задачи объединяет необходимость работы с контекстом и извлечения информации, сформированной при обработке одной части данных, для обработки других частей.

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L04/sequence_data.png" width="700"></center>

<center><em>Source: <a href="https://cbare.github.io/2019-01-27/deep-learning-sequence-models.html">Deep Learning - Sequence Models</a></em></center>

### Рекуррентная ячейка и рекуррентный слой

Рассмотрим работу рекуррентных сетей применительно к текстовым данным.

Каждое слово $x_t$ в предложении последовательно обрабатывается с помощью **ячейки RNN**.

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L04/RNN_cell.png" width="700"></center>

<center><em>Source: <a href="http://vbystricky.ru/2021/05/rnn_lstm_gru_etc.html">RNN, LSTM, GRU</a></em></center>

1. На вход ячейки поступает входной вектор $x_t$ —  эмбеддинг слова с индексом $t$. Он имеет фиксированный размер $k$.

2. Ячейка принимает еще один параметр $h_{t-1}$ — **скрытое состояние** или **память** (hidden state). Это вектор, хранящий информацию о предшествующем контексте. Он тоже имеет фиксированный размер $n$.

3. Вектор $x_t$ умножается на матрицу $W^{nk}$ (размер $n \times k$), которая содержит обучаемые веса: $W^{nk} \cdot x_t$. Получаем новый вектор размера $n$.

4. Вектор $h_{t-1}$ умножается на другую матрицу весов $W^{nn}$ (размер $n \times n$): $ W^{nn} \cdot h_{t-1} $. Получаем новый вектор размера $n$.

5. Получившиеся векторы имеют одинаковый размер, их можно сложить: $W^{nk} \cdot x_t + W^{nn} \cdot h_{t-1}$.

6. К получившемуся вектору применим функцию активации — гиперболический тангенс: $tanh(W^{nk} \cdot x_t + W^{nn} \cdot h_{t-1})$. Это и будет новым скрытым состоянием $h_t$. Оно зависит от предыдущего состояния $h_{t-1}$ и текущего элемента последовательности $x_t$.

7. Рассчитанное скрытое состояние $h_t$ является представлением текущего слова $x_t$ с учетом предшествующего контекста и передается в качестве старого скрытого состояния для следующего слова $x_{t+1}$.

Для обработки всей последовательности слов используется **слой RNN**, где ячейка применяется ко всем словам в цикле.

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L04/RNN_layer.png" width="700"></center>

<center><em>Source: <a href="http://vbystricky.ru/2021/05/rnn_lstm_gru_etc.html">RNN, LSTM, GRU</a></em></center>

Когда первый токен $x_0$ подается в ячейку, вектор памяти $h$ инициализируется нулями.

Вектор памяти $h_0$ для первого токена $x_0$ передается в следующую ячейку, обрабатывающую второй токен $x_1$. Вектор $h_0$ также является контекстуализированным представлением токена $x_0$ — $y_0$.

Для каждого последующего токена вектор памяти учитывает контекст из предыдущих токенов.

Вектор $y_t$ является не только контекстуализированным представлением для последнего токена $x_t$, но и вектором всего предложения, т.к. содержит информацию обо всех токенах.

В зависимости от типа задачи, можно использовать контекстуализированные вектора для каждого слова или только вектор всего предложения (последнего слова).

## RNN слой в PyTorch

В PyTorch есть слой — `torch.nn.RNN` [🛠️[doc]](https://pytorch.org/docs/stable/generated/torch.nn.RNN.html), который реализует логику, описанную выше.

Также есть сущность `torch.nn.RNNCell` [🛠️[doc]](https://pytorch.org/docs/stable/generated/torch.nn.RNNCell.html), которая реализует вычисления на одном такте времени.

Слой `nn.RNN` фактически является оберткой, которая вызывает `nn.RNNCell` в цикле по длине входной последовательности.

Параметры слоя `nn.RNN`:

* **`input_size`** — размерность $\large x_t$, целое число.

* **`hidden_size`** — размерность $\large h_t$, целое число. Фактически это количество нейронов в рекуррентном слое.


Техническая особенность рекуррентных слоев в PyTorch: по умолчанию ожидаемые размерности входа такие:

**`[длина последовательности, размер батча, размерность входа]`**

Однако если при создании слоя указать `batch_first=True`, то можно подавать входные значения в более привычном формате, когда размер батча стоит на первом месте:

**`[размер батча, длина последовательности, размерность входа]`**



In [ ]:
import torch

torch.manual_seed(42)

rnn = torch.nn.RNN(input_size=3, hidden_size=2, batch_first=True)

dummy_batched_seq = torch.randn((16, 57, 3))  # batch_size, seq_len, input_size
out, h = rnn(dummy_batched_seq)

print("Input shape:".ljust(20), f"{dummy_batched_seq.shape}")
print("Out shape:".ljust(20), f"{out.shape}")
print("Last hidden state shape:".ljust(20), f"{h.shape}")

При вызове слой возвращает два объекта:
* `out` — последовательность скрытых состояний,
* `h` — скрытое состояние на последнем такте.

Мы указали `batch_first=True`, при этом `out` сохранил последовательность размерностей, как у входа, а вот у `h` размерность батча встала на второе место.

In [ ]:
h_batch_first = h.permute(1, 0, 2)

print(f"h is last out: {(h_batch_first == out[:, -1:, :]).all().item()}")

### Многослойные RNN

RNN-блоки можно объединять в слои, накладывая их друг на друга. Для этой операции в `torch.nn.RNN` есть аргумент `num_layers`, с помощью которого можно указать количество слоёв.

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-content/L04/out/rnn_multiple_layers.png" width="500"></center>

In [ ]:
dummy_input = torch.randn((16, 6, 3))  # batch_size, seq_len, input_size
rnn = torch.nn.RNN(input_size=3, hidden_size=2, num_layers=2, batch_first=True)

out, h = rnn(dummy_input)

print()
print("Out:\n", out.shape)  # Hidden states for all elements from top layer
print("h:\n", h.shape)  # Hidden states for last element for all layers

## Проблемы RNN

Теоретически, можно было бы сразу пропустить все данные через сеть и затем вычислить градиент, однако возникнут следующие проблемы:

 - большие последовательности не поместятся в памяти,
 - так как цепочка будет очень длинной, возникнет затухание/взрыв градиента,
 - по мере прохождения сигнала по цепи контекст затирается.


Допустим, у нас есть длинная последовательность. Если мы сразу предсказываем, то в каждый момент времени нужно распространить Loss. И все ячейки нужно обновить во время backpropogation. Все градиенты нужно посчитать. Возникают проблемы, связанные с нехваткой памяти.

Есть специальные тесты для проверки, контекст какой длины использует RNN при предсказании. Если мы делаем предсказание только в последней ячейке, может оказаться, что используется, скажем, информация только о последних 10 словах предложения.

Функция активации Tanh постепенно затирает контекст.

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L04/backprop_through_time.png" width="700"><center>

<center><em>Source: <a href="http://cs231n.stanford.edu/slides/2021/lecture_10.pdf">CS231n: Recurrent Neural Network</a></em></center>

Затухающий/взрывающийся градиент (Vanishing/exploding gradient) — явления затухающего и взрывающегося градиента часто встречаются в контексте RNN. И при большой длине последовательности это становится критичным. Причина в том, что зависимость величины градиента от числа слоёв экспоненциальная, поскольку веса умножаются многократно.

$dL ∝ (W)^N:$

$W > 1 \rightarrow$ взрыв, $W < 1 \rightarrow$ затухание.

<img src ="https://edunet.kea.su/repo/EduNet_NLP-content/L04/out/simple_rnn_backprop.png" width="1000">

Один из путей решения проблемы — **градиентное отсечение** (Gradient truncating) — метод, который ограничивает максимально допустимое значение градиента, позволяя избежать градиентного взрыва.

А от затухания градиента может помочь **пропускание** **градиента по частям**, на сколько-то шагов по времени назад или вперёд, а не через всю нейросеть. Да, градиент будет не совсем точно считаться, и мы будем терять в качестве. Но это экономит память.

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L04/truncated_backprop.png"  width="700"></center>

<center><em>Source: <a href="http://cs231n.stanford.edu/slides/2021/lecture_10.pdf">CS231n: Recurrent Neural Network</a></em></center>

## LSTM (Long Short-Term Memory)

Обычная RNN имела множество проблем, в том числе в ней очень быстро затухала информация о предыдущих элементах последовательности. Помимо этого были проблемы с затуханием/взрывом градиента.

Эти проблемы были частично решены в LSTM, предложенной в [Long Short-Term Memory (Hochreiter & Schmidhuber, 1997) 🎓[article]](http://www.bioinf.jku.at/publications/older/2604.pdf).

В обычной RNN-ячейке был только один путь передачи информации. На каждом шаге мы конкатенировали предыдущее скрытое состояние с текущим входом и пропускали их через линейный слой и активацией:

<div align="center">
<html>
<head>
<style>
table, th, td {
  border: 1px solid black;
  border-collapse: collapse;
}
<font size="2" face="Times New Romans" >


</style>
</head>
<body>

<table >

<tr>
<td>

<center><img src = "https://edunet.kea.su/repo/EduNet_NLP-content/L04/out/simple_rnn_h_state.png" width="500"></center>

</td>
<td>

<table >
<tr>
<td>

$\large h_t = \tanh(W \cdot [h_{t-1}, x_t])$

</td>

</tr>


</table>

</td>
</tr>

</table>








При этом информация о предыдущих элементах последовательности очень быстро затухает и теряется общая информация о предложении.

Структура ячейки LSTM намного сложнее. Здесь есть целых 4 линейных слоя, каждый из которых выполняет разные задачи.

<div align="center">
<html>
<head>
<style>
table, th, td {
  border: 1px solid black;
  border-collapse: collapse;
}
<font size="2" face="Times New Romans" >


</style>
</head>
<body>

<table >
<tr>
<td>

<center><img src="https://edunet.kea.su/repo/EduNet_NLP-content/L04/out/lstm_chain.png" width="500"></center>
<center><img src="https://edunet.kea.su/repo/EduNet_NLP-content/L04/out/lstm_chain_notation.png" width="700"></center>

</td>
<td>

<table >
<tr>
<td>

$\large f_t = σ(W_f \cdot [h_{t-1}, x_t])\ \ \ \ $

</td>
<td>

$$\large \text{forget  gate}$$

</td>
</tr>

<tr>
<td>

$\large i_t = σ(W_i \cdot [h_{t-1}, x_t])$

</td>
<td>

$$\large \text{input gate}$$

</td>
</tr>

<tr>
<td>

$\large o_t = σ(W_o \cdot [h_{t-1}, x_t])$

</td>
<td>

$$\large \text{output gate}$$

</td>
</tr>

<tr>
<td>

$\large c^\prime_t = \tanh(W_c \cdot [h_{t-1}, x_t])$

</td>
<td>

$$\large \text{candidate cell state}$$

</td>
</tr>

<tr>
<td>

$\large c_t = f_t\otimes c_{t-1} + i_t \otimes c^\prime_t$

</td>
<td>

$$\large \text{cell state}$$

</td>
</tr>

<tr>
<td>

$\large h_t = o_t\otimes \tanh(c_t)$

</td>
<td>

$$\large  \text{hidden state}$$

</td>
</tr>
</table>

</td>
</tr>

</table>








Главное нововведение: в LSTM добавлен путь $c$, который по задумке должен этот общий контекст сохранять.

<center><img src="https://edunet.kea.su/repo/EduNet_NLP-content/L04/out/lstm_c_state_highway.png" width="500"></center>

Другими словами, путь $c$ (cell state, иногда называется highway, магистраль)  помогает нейросети сохранять важную информацию, встретившуюся в какой-то момент в последовательности в прошлом, все время, пока эта информация требуется.

По формулам также видно, как возросла сложность.

Отличие от RNN состоит в том, что кроме $h$ возвращается еще и $c$.

In [ ]:
import torch.nn as nn


lstm = nn.LSTM(input_size=3, hidden_size=2, batch_first=True)
input = torch.randn(16, 57, 3)  # batch_size, seq_len, input_size
out, (h, c) = lstm(input)  # h and c returned in tuple

print("Input shape:".ljust(15), input.shape)
print("Shape of h".ljust(15), h.shape)  # 1, batch_size, hidden_size
print("Shape of c".ljust(15), c.shape)  # 1, batch_size, hidden_size
print("Output shape:".ljust(15), out.shape)  # batch_size, seq_len, hidden_size

## GRU (Gated Recurrent Unit)

Самая известная модификация LSTM — GRU. Она более компактна за счет сильных упрощений в сравнении со стандартной LSTM.

Главные изменения: объединены forget и input gates, слиты $h_t$ и $c_t$, которые в обычной LSTM только участвовали в формировании друг друга.

<div align="center">
<html>
<head>
<style>
table, th, td {
  border: 1px solid black;
  border-collapse: collapse;
}
<font size="2" face="Times New Romans" >


</style>
</head>
<body>

<table >
<tr>
<td>

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-content/L04/out/gru_basic_block.png" width="500"></center>

</td>
<td>

<table >
<tr>
<td>

$\large z_t = \sigma(W_z \cdot [h_{t-1}, x_t])$

</td>

</tr>

<tr>
<td>

$\large r_t = \sigma(W_r \cdot [h_{t-1}, x_t])$

</td>

</tr>

<tr>
<td>

$\large \tilde h_t = \tanh(W \cdot [r_t \otimes h_{t-1}, x_t])$

</td>

</tr>

<tr>
<td>

$\large h_t = (1-z_t) \otimes h_{t-1} + z_t \otimes \tilde h_t$

</td>


</table>








</td>
</tr>

</table>








In [ ]:
gru = nn.GRU(input_size=3, hidden_size=2, batch_first=True)
input = torch.randn(16, 57, 3)  # batch_size, seq_len, input_size
out, h = gru(input)

print("Input shape:".ljust(15), input.shape)
print("Shape of h:".ljust(15), h.shape)  # 1, batch_size, hidden_size
print("Output shape:".ljust(15), out.shape)  # batch_size, seq_len, hidden_size

Практический опыт исследователей: иногда лучше работает GRU, иногда — LSTM. Точный рецепт успеха сказать нельзя.

## Типы задач

- Один к одному: на вход подается один элемент, на выходе получаем вероятность класса — классификация изображений.

- Один ко многим: на вход приходит один элемент, а на выходе получаем целую последовательность — генерация текста по слову, текстовой подписи по изображению.

- Многие к одному: на вход сети подается последовательность, а в качестве выхода получаем один вектор вероятностей для классов — классификация текстов (анализ тональности, определение языка).

- Многие ко многим: преобразовать последовательность в последовательность
  - количество выходов равно количеству входов — тегирование (именованные сущности, части речи).
  - количество выходов сети не обязательно равно количеству входов — машинный перевод, суммаризация.

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L04/RNN_tasks.png" width="800"></center>

<center><em>Source: <a href="https://dotnettutorials.net/lesson/recurrent-neural-network/">Recurrent Neural Network (RNNs)</a></em></center>

## RNN для машинного перевода

Машинный перевод относится к задаче преобразования одной последовательности в другую, длина которых может быть любой и не обязательно должна совпадать: "многие ко многим" или sequence-to-sequence (сокращенно seq2seq).

Модель для машинного перевода состоит из блока <font color="blue"> кодировщика </font> и <font color="red"> декодировщика</font>. Ниже представлена более подробная схема.

<center><img src="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L04/seq2seq.png" width="600"></center>

<center><em>Source: <a href="https://blog.paperspace.com/introduction-to-neural-machine-translation-with-bahdanaus-attention/">Introduction to Neural Machine Translation</a></em></center>

<div align="center">
    <table >
     <tr>
       <td>

Архитектура <font color="blue">кодировщика</font>:

- на вход первой ячейки RNN поступает вектор слова "I"<br>
и нулевое скрытое состояние,
- вектор слова "I" обрабатывается внутри первой ячейки RNN;
- на вход второй ячейки поступает вектор слова "am"<br>и измененный вектор слова "I" как скрытое состояние;
- вектор слова "am" обрабатывается внутри второй ячейки RNN;
- $\cdots$
- вектор слова "student" обрабатывается внутри четвертой ячейки RNN;
- измененный вектор слова "student" (= вектор предложения)<br>поступает на вход декодера как скрытое состояние.

</td>

<td>

Архитектура <font color="red">декодировщика</font>:
- на вход первой ячейки RNN поступает<br>вектор спецсимвола начала предложения <br>и вектор исходного предложения из энкодера как скрытое состояние;
-  вектор спецсимвола начала предложения<br>обрабатывается внутри первой ячейки RNN;
- на вход второй ячейки поступает вектор слова "Je"<br>и измененный вектор спецсимвола начала как скрытое состояние;
- вектор слова "Je" обрабатывается внутри второй ячейки RNN;
- $\cdots$
- вектор слова "étudiant" обрабатывается внутри четвертой ячейки RNN;
- измененные векторы каждого слова<br>передаются на линейный слой, применяется softmax;
- на выходе получаем 4 вектора, длина которых равна длине словаря, —<br>это распределение вероятностей для следующего элемента<br>при условии текущей последовательности.


</td>
     </tr>
    </table>
    </div>

Итак,
- <font color="blue">кодировщик</font> используется для получения конекстуализированного вектора исходного предложения;
- <font color="red">декодировщик</font> используется для генерации целевого предложения;
- вектор исходного предложения используется в качестве скрытого состояния на первом шаге генерации.

**Проблема Sequence-to-Sequence**: использование только конечного скрытого состояния энкодера для представления всей входной последовательности приводит к потере информации, особенно с начала последовательности;
- по мере того, как энкодер обрабатывает входную последовательность, ожидается, что в конечном скрытом состоянии будет собрана вся необходимая информация;
- когда последовательность становится длиннее, этому единственному состоянию труднее сохранять всю необходимую информацию из предыдущих частей последовательности.

Чтобы модель могла фокусироваться на различных частях входной последовательности и сохранять больше информации на протяжении всего процесса кодирования и декодирования, был предложен механизм внимания (attention mechanism).

[[paper] 🎓 Bahdanau D., Cho K., Bengio Y. (2014). Neural Machine Translation by Jointly Learning to Align and Translate](https://arxiv.org/abs/1409.0473)

<center><img src="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L04/seq2seq_attention.png" width="600"></center>

<center><em>Source: <a href="https://blog.paperspace.com/introduction-to-neural-machine-translation-with-bahdanaus-attention/">Introduction to Neural Machine Translation</a></em></center>

Архитектура энкодера не меняется, все изменения касаются архитектуры декодера.
- на вход первой ячейки RNN также поступает вектор спецсимвола начала предложения и вектор исходного предложения из энкодера как скрытое состояние;
-  вектор спецсимвола начала предложения обрабатывается внутри первой ячейки RNN;
- для вектора спецсимвола начала предложения и векторов каждого слова исходной последовательности считается мера сходства — это и есть веса внимания (attention weights);
- если векторы некоторого слова целевого и исходного предложения схожи, то при переводе (генерации) данного слова декодер больше "обращает внимания" на него;
- вектор каждого слова исходного предложения умножается на свой вес внимания, затем все векторы складываются, получаем контекстный вектор (context vector);
- вектор спецсимвола начала предложения и контекстный вектор конкатенируются;
- конкатенированный вектор передается на линейный слой, применяется softmax;
- получаем вектор, длина которого равна длине словаря, — это распределение вероятностей для следующего элемента при условии спецсимвола начала предложения;
- на вход второй ячейки поступает вектор слова "Je" и измененный вектор спецсимвола начала как скрытое состояние;
- вектор слова "Je" обрабатывается внутри второй ячейки RNN;
- считаются веса внимания для слова "Je" и всех слов целевого предложения;
- $\cdots$
- для каждого слова получаем распределение вероятностей.

<center><img src="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L04/attention_equation.jpg" width="700"></center>

<center><em>Source: <a href="https://www.google.com/url?sa=i&url=https%3A%2F%2Fai.plainenglish.io%2Fintroduction-to-attention-mechanism-bahdanau-and-luong-attention-e2efd6ce22da&psig=AOvVaw1NlMR6N0XSxl6zYQwhAlYw&ust=1723824501708000&source=images&cd=vfe&opi=89978449&ved=0CAUQtaYDahcKEwiAhL3usPeHAxUAAAAAHQAAAAAQDw">Introduction to Attention Mechanism</a></em></center>

Механизм внимания позволил исправить некоторые проблемы применения рекуррентных сети, однако ряд недостатков остается:
- не способны запоминать дистантные зависимости
- возникает взрыв и затухание градиента
- вычисления проводятся последовательно и занимают много времени

Что если убрать рекуррентность и оставить только механизм внимания?

# Архитектура Трансформер

Архитектура, в которой отсутствует рекуррентность и используется только механизм внимания, получила название Трансформер.

[[paper] 🎓 Vaswani A. et al. (2017).Attention Is All You Need](https://arxiv.org/abs/1706.03762)

Как и в случае с рекуррентной сетью seq2seq, Трансформер состоит из блока энкодера и блока декодера.
- Энкодер обрабатывает исходную последовательность и кодирует ее.
- Декодер обрабатывает целевую последовательность с учетом информации из энкодера. Выход из декодера является предсказанием модели.

Блок энкодера состоит из 6 энкодеров, расположенных друг за другом. Блок декодера – это стек декодеров, представленных в том же количестве.

<center><img src="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L04/transformer.jpg" width="860"></center>

<center><em>Source: <a href="https://jalammar.github.io/illustrated-transformer/">The Illustrated Transformer</a></em></center>

Все энкодеры идентичны по структуре, хотя и имеют разные веса. Каждый можно разделить на два подслоя.
- Входная последовательность, поступающая в энкодер, сначала проходит через слой внутреннего внимания (self-attention), помогающий энкодеру посмотреть на другие слова во входном предложении во время кодирования конкретного слова.
- Выход слоя внутреннего внимания отправляется в нейронную сеть прямого распространения (feed-forward neural network).

Декодер также содержит эти два слоя, но между ними есть слой внимания, который помогает декодеру фокусироваться на релевантных частях исходного предложения (аналогично механизму внимания в моделях seq2seq).

<center><img src="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L04/enc_dec_tr.png" width="650"></center>

<center><em>Source: <a href="https://jalammar.github.io/illustrated-transformer/">The Illustrated Transformer</a></em></center>

Как и в случае других алгоритмов, каждое слово входной и целевой последовательности преобразуется в вектор.

После того как слова входящего предложения преобразовались в эмбеддинги, каждый из них по отдельности проходит через слои энкодера. Одна из основных особенностей Трансформера состоит в том, что каждое слово идет по своей собственной траектории.

<center><img src="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L04/encoder.png" width="700"></center>

<center><em>Source: <a href="https://jalammar.github.io/illustrated-transformer/">The Illustrated Transformer</a></em></center>

Мы попробуем создать и обучить модель архитектуры Трансформер, основываясь на тьюториале:

[[doc] 🛠️ The Annotated Transformer](https://nlp.seas.harvard.edu/annotated-transformer/)

## Создание модели

### Механизм внутреннего внимания

Пусть мы хотим перевести предложение: *The animal didn't cross the street because it was too tired*. Местоимение *it* может относиться к улице (*street*) или к животному (*animal*). Когда модель обрабатывает слово *it*, слой внутреннего внимания помогает понять, что *it* относится к *animal*.

По мере того как модель обрабатывает каждое слово входной последовательности, внутреннее внимание позволяет "взглянуть" на другие слова и лучше закодировать данное слово. Механизм внутреннего внимания – это метод, который Трансформер использует, чтобы смоделировать "понимание" других релевантных слов при обработке конкретного слова.

Во время кодирования *it* в энкодере №5 часть механизма внимания фокусируется на *The animal* и использует фрагмент его представления для кодирования *it*.

<center><img src="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L04/self_attention.png" width="400"></center>

<center><em>Source: <a href="https://jalammar.github.io/illustrated-transformer/">The Illustrated Transformer</a></em></center>

#### На примере векторов

При вычислении внутреннего внимания используются: вектор запроса $query$, вектор ключа $key$ и вектор значения $value$. Они создаются с помощью умножения эмбеддинга слова последовательности на три матрицы весов (линейных слоя) $W^Q, W^K, W^V$. Размер новых векторов – 64, размер исходных векторов – 512.

$q_i=x_iW^Q$

$k_i=x_iW^K$

$v_i=x_iW^V$

<center><img src="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L04/vector_attention.png" width="700"></center>

<center><em>Source: <a href="https://jalammar.github.io/illustrated-transformer/">The Illustrated Transformer</a></em></center>

Далее вычисляется коэффициент внутреннего внимания $score$ для $i$-го слова по отношению к каждому слову в предложении. Коэффициент определяет, насколько нужно сфокусироваться на других частях предложения во время кодирования слова в $i$-й позиции. Коэффициент подсчитывается с помощью скалярного произведения вектора запроса $q$ $i$-го слова и вектора ключа $k$ каждого слова.

$score_{ij}=q_i \cdot k_j$

📌 Какие векторы нужно перемножить, чтобы посчитать коэффициент внимания слова *Thinking* по отношению к слову *Machines*? По отношению к самому себе?

На следующем шаге коэффициенты делятся на квадратный корень из $d_k$ – размера векторов ключа $k$. К получившимся значениям применяется функция активации softmax, чтобы коэффициенты в сумме давали 1. Полученный софтмакс-коэффициент определяет, в какой мере каждое из слов предложения "фокусируется" на другом слове.

$softmax.score_{ij}=softmax(\frac{score_i}{\sqrt d_k})$

После этого каждый вектор значения $v$ умножается на софтмакс-коэффициент, получаем взвешенные векторы. Идея в том, что нужно сохранять без изменений значения слов, на которых мы фокусируемся, и отвести на второй план нерелевантные слова (умножив их на небольшие значения, например, 0.001). Затем  взвешенные векторы складываются. Результат (взвешенная сумма) представляет собой выход слоя внутреннего внимания для $i$-го слова.

$sum_i=\sum_{j=1}^nv_j \cdot softmax.score_{ij}$

Полученный вектор передается дальше в нейронную сеть прямого распространения.

<center><img src="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L04/attention_example.png" width="600"></center>

<center><em>Source: <a href="https://jalammar.github.io/illustrated-transformer/">The Illustrated Transformer</a></em></center>

#### На примере матриц

Аналогично, матрицы запроса $Q$, ключа $K$ и значения $V$ вычисляются с помощью умножения эмбеддингов матрицы $X$ на матрицы весов (линейные слои) $W^Q, W^K, W^V$. Каждая строка в матрице $X$ соответствует слову в предложении.

<center><img src="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L04/matrix_attention.png" width="450"></center>

<center><em>Source: <a href="https://jalammar.github.io/illustrated-transformer/">The Illustrated Transformer</a></em></center>

Последующие этапы для вычисления выхода слоя внутреннего внимания могут быть отражены в одной формуле.

<center><img src="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L04/attention_score.png" width="600"></center>

<center><em>Source: <a href="https://jalammar.github.io/illustrated-transformer/">The Illustrated Transformer</a></em></center>

Напишем функцию `attention` для подсчета внутреннего внимания.

$Attention(Q, K, V) = softmax\large(\frac{QK^T}{\sqrt{d_k}})V$

Для умножения матриц $Q, K, V$ используется метод `torch.matmul()`.

Транспонирование осуществляется с помощью метода `.transpose`.

In [ ]:
import math, torch
import numpy as np
import torch.nn.functional as F

def attention(query, key, value, mask=None, dropout=None):
    """
    Compute 'Scaled Dot Product Attention'
    """
    d_k = query.size(-1)
    # compute the attention scores by using torch.matmul
    scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    p_attn = F.softmax(scores, dim = -1)
    if dropout is not None:
        p_attn = dropout(p_attn)
    # compute the result as the values weighted by attention probabilities (again, using torch.matmul)
    result = torch.matmul(p_attn, value)
    return result, p_attn

In [ ]:
results, attentions = attention(
    torch.tensor([[0, 0], [0, 1], [1, 1]], dtype=torch.float),
    torch.tensor([[100, 0], [0, 100], [0, 0]], dtype=torch.float),
    torch.tensor([[1, 0], [0, 1], [0, 0]], dtype=torch.float),
)
print(results)
print(attentions)

assert np.allclose(results[0].numpy(), [1/3, 1/3])  # the first query attends to all keys equally
assert np.allclose(results[1].numpy(), [0, 1])      # the second query attends only to the second key
assert np.allclose(results[2].numpy(), [1/2, 1/2])  # the third query attends to the first and second key equally

### Множественное внимание

В Трансформере механизм внутреннего внимания совершенствуется с помощью добавления множественного внимания (multi-headed attention). Это повышает способность модели фокусироваться на разных словах. В случае множественного внимания мы располагаем отдельными матрицами весов $W^Q, W^K, W^V$  для каждой "головы", что в результате дает разные матрицы $Q, K, V$. Как и ранее, матрица $X$ умножается на веса $W^Q, W^K, W^V$ для получения матриц $Q, K, V$.

Трансформер использует 8 "голов" внимания, так что в итоге у нас получается 8 наборов для каждого энкодера и декодера. Сделав те же вычисления внутреннего внимания 8 раз с разными матрицами весов, в результате получим 8 разных матриц.

<center><img src="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L04/multihead_attention.png" width="600"></center>

<center><em>Source: <a href="https://jalammar.github.io/illustrated-transformer/">The Illustrated Transformer</a></em></center>

Однако слой сети прямого распространения не ожидает, что к нему поступит 8 матриц – он ждет всего одну, в которую нам и необходимо сжать полученные матрицы. Чтобы это сделать, можно конкатенировать и затем умножить их на дополнительные веса матрицы $W^O$.

<center><img src="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L04/concatenate.png" width="800"></center>

<center><em>Source: <a href="https://jalammar.github.io/illustrated-transformer/">The Illustrated Transformer</a></em></center>

Мы можем отобразить все "головы" внимания на одной картинке.

📌 Какие головы больше фокусируются на каких словах?

<center><img src="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L04/multihead_viz.png" width="400"></center>

<center><em>Source: <a href="https://jalammar.github.io/illustrated-transformer/">The Illustrated Transformer</a></em></center>

Для создание множественного внимания необходим класс `MultiHeadedAttention`.

Каждая голова внимания независимо вычисляет внимание (скалярное произведение) для своих матриц $Q,K,V$. Следовательно, каждая голова может "обращать внимание" на разные части предложения.

$MultiHead(Q, K, V) = Concat(head_1, ..., head_h)W^O$

In [ ]:
import copy
def clones(module, N):
    "Produce N identical layers"
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

In [ ]:
import torch.nn as nn

class MultiHeadedAttention(nn.Module):
    def __init__(self, h, d_model, dropout=0.1):
        "Take in model size and number of heads"
        super(MultiHeadedAttention, self).__init__()
        assert d_model % h == 0
        # We assume d_v always equals d_k
        self.d_k = d_model // h
        self.h = h
        self.linears = clones(nn.Linear(d_model, d_model), 4)
        self.attn = None
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, query, key, value, mask=None):
        if mask is not None:
            # Same mask applied to all h heads.
            mask = mask.unsqueeze(1)
        nbatches = query.size(0)

        # 1) Do all the linear projections in batch from d_model => h x d_k
        query, key, value = \
            [l(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
             for l, x in zip(self.linears, (query, key, value))]

        # 2) Apply attention on all the projected vectors in batch
        x, self.attn = attention(query, key, value, mask=mask,
                                 dropout=self.dropout)

        # 3) "Concat" using a view and apply a final linear
        x = x.transpose(1, 2).contiguous() \
             .view(nbatches, -1, self.h * self.d_k)
        return self.linears[-1](x)

### Кодирование позиции

Из-за того, что мы избавили от рекуррентной нейронной сети, информация о порядке слов перестала учитываться моделью. В классе `MultiHeadAttention` операции применяются к размерности 2 (признаки слов), но не к размерности 1 (слова последовательности).

Для того чтобы модель понимала порядок слов, вводятся векторы кодирования позиции (positional encoding). Они суммируются с векторами каждого слова.

<center><img src="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L04/positional_encoding.png" width="850"></center>

<center><em>Source: <a href="https://jalammar.github.io/illustrated-transformer/">The Illustrated Transformer</a></em></center>

Информация о позиции кодируется с помощью тригонометрических функций: синуса и косинуса.

Пусть есть входная последовательность длины $L$, нужно закодировать позицию некоторого элемента $pos$.

$pos$ — индекс слова в исходной последовательности, $0 \leq pos < L$

$d$ — размер эмеддингов входного слоя

$i$ — номер координаты эмбеддинга входного слоя, $0 \leq i < \large \frac{d}{2}$

$p_0$ — первое слово последовательности, $d$ — длина вектора, $i$ — координаты вектора.

<center><img src="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L04/pe1.png" width="500"></center>

<center><em>Source: <a href="https://www.youtube.com/watch?v=dichIcUZfOw">Visual Guide to Transformer Neural Networks</a></em></center>

Слова с разным номером позиции будут иметь разные значения значения на оси $y$. Однако для некоторых позиций ($p_0$ и $p_6$) значения совпадают, поскольку синус — периодическая функция.

<center><img src="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L04/pe2.png" width="700"></center>

<center><em>Source: <a href="https://www.youtube.com/watch?v=dichIcUZfOw">Visual Guide to Transformer Neural Networks</a></em></center>

Получить разные значения можно благодаря параметру $i$. При изменении значения $i$ меняется частота, что приводит к получению разных значений для $p_0$ и $p_6$.

<center><img src="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L04/pe3.png" width="800"></center>

<center><em>Source: <a href="https://www.youtube.com/watch?v=dichIcUZfOw">Visual Guide to Transformer Neural Networks</a></em></center>

Создадим класс `PositionalEncoding` для кодирования позиции.

$PE_{(pos, 2i)}=sin(pos/10000^{2i/d_{model}})$

$PE_{(pos, 2i+1)}=cos(pos/10000^{2i/d_{model}})$

In [ ]:
from torch.autograd import Variable

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + Variable(self.pe[:, :x.size(1)], requires_grad=False)
        return self.dropout(x)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize=(15, 5))
pe = PositionalEncoding(20, 0)
y = pe.forward(Variable(torch.zeros(1, 100, 20)))
plt.plot(np.arange(100), y[0, :, 4:8].data.numpy())
plt.legend(["dim %d"%p for p in [4,5,6,7]])
None

### Слои Embedding и преобразование Softmax

Аналогично другим моделям преобразования последовательностей, мы используем обученные эмбеддинги для преобразования входных и выходных токенов в векторы размерности $d_{\text{model}}$. Мы также используем линейное преобразование и функцию softmax для преобразования выходных данных декодера в прогнозируемые вероятности следующего токена. В нашей модели мы используем одну и ту же весовую матрицу между слоями эмбеддингов и линейным преобразованием, выполненным до softmax. В слоях эмбеддингов мы умножаем эти веса на $\sqrt{d_{\text{model}}}$.

In [ ]:
class Embeddings(nn.Module):
    def __init__(self, d_model, vocab):
        super(Embeddings, self).__init__()
        self.lut = nn.Embedding(vocab, d_model)
        self.d_model = d_model

    def forward(self, x):
        return self.lut(x) * math.sqrt(self.d_model)

### Cеть прямого распространения

За слоем множественного внимания в энкодере и декодере следует сеть прямого распространения. Она состоит из двух линейных слоев и функции активации ReLU между ними. Размер входных и выходных данных — 512, промежуточный размер — 2048 (в 4 раза больше).

In [ ]:
class PositionwiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.w_2(self.dropout(F.relu(self.w_1(x))))

### Энкодер

Можем перейти к реализации блока энкодера. Он состоит из стека $N=6$ идентичных слоев.

<center><img src="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L04/enc_details.png" width="400"></center>

<center><em>Source: <a href="https://jalammar.github.io/illustrated-transformer/">The Illustrated Transformer</a></em></center>

In [ ]:
class Encoder(nn.Module):
    "Core encoder is a stack of N layers"
    def __init__(self, layer, N):
        super(Encoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)

    def forward(self, x, mask):
        "Pass the input (and mask) through each layer in turn"
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)

Каждый подслой (внутреннее внимание, полносвязный) включает остаточное соединение (residual connection), за которым следует этап нормализации слоя ([[doc] 🛠️ layer normalization](https://pytorch.org/docs/stable/generated/torch.nn.LayerNorm.htm)).



<center><img src="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L04/connection_normalization.png" width="500"></center>

<center><em>Source: <a href="https://jalammar.github.io/illustrated-transformer/">The Illustrated Transformer</a></em></center>

Эмбеддинг каждого элемента последовательности дублируется: одна копия поступает на вход слоя (внимания или сети прямого распространения), другая копия не меняется и прибавляется к выходу слоя. Это позволяет избежать затухания градиента.

После применяется нормализация. Веса могут принимать значения в разном диапазоне, но скорость обучение одинаковая. Из-за этого возникает сложность с обновлением весов.

Нормализация применяется по предложениям: применяется к первому признаку каждого слова в первом предложении, затем ко второму признаку каждого слова в первом предожении и т.д.

Среднее: $\mu_i =\frac{1}{m}\sum_{j=1}^mx_{ij}$

Стандартное отклонение: $\sigma_i = \frac{1}{m}\sum_{j=1}^m(x_{ij}-\mu_i)$

Нормализованное значение: $\hat x_{ij} = \frac {x_{ij}-\mu_i}{\sigma_i}$

<center><img src="https://edunet.kea.su/repo/EduNet_NLP-content/L04/out/layer_normalization.png" width="400"></center>

Мы используем остаточное соединение вокруг каждого из двух подслоев, далее следует нормализация слоя.

In [ ]:
class LayerNorm(nn.Module):
    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        """
        Return a2 * x_normalized + b2,
        where x_normalized is calculated by subtracting row-wise means from x and dividing the result by row-wise standard deviation + eps.
        standard deviation is calculated with Bessel's correction (the default in Pytorch)
        """
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

In [ ]:
ln = LayerNorm(2, eps=0)
with torch.no_grad():
    result = ln(torch.tensor([[0.0, 1], [100, 101], [100, 200]])).numpy()

# becasue of Bessel's correction, standard deviation is pulled to 0. Here we un-pull it back.
print(result)
result_unnormalized = result / np.sqrt(0.5)
assert (result_unnormalized[:, 0] == -1).all()
assert (result_unnormalized[:, 1] == 1).all()

То есть результат каждого подслоя — $\mathrm{LayerNorm}(x + \mathrm{Sublayer}(x))$, где $\mathrm{Sublayer}(x)$ — это функция, реализуемая самим подслоем. Мы применяем dropout к выходным данным каждого подслоя, прежде чем они складываются и нормализуются.

Чтобы упростить эти остаточное соединение, все подслои в модели, а также слой эмбеддингов выдают выходные данные с размерностью $d_{\text{model}}=512$.

In [ ]:
class SublayerConnection(nn.Module):
    """
    A residual connection followed by a layer norm.
    Note for code simplicity the norm is first as opposed to last.
    """
    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        "Apply residual connection to any sublayer with the same size."
        return x + self.dropout(sublayer(self.norm(x)))

Каждый слой состоит из двух подслоев. Первый представляет собой механизм множественного внимания, а второй — полносвязную сеть прямого распространения.

In [ ]:
class EncoderLayer(nn.Module):
    "Encoder is made up of self-attn and feed forward"
    def __init__(self, size, self_attn, feed_forward, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 2)
        self.size = size

    def forward(self, x, mask):
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask))
        return self.sublayer[1](x, self.feed_forward)

### Декодер

Блок декодера устроен очень похоже, но есть несколько отличий.
- Декодер принимает два аргумента: целевое предложение и выход энкодера.
- Используется два слоя множественного внимания
  - маскированное множественное внимания для обработки целевого предложения
  - множественное внимание между энкодером и декодером для сравнения целевого и исходного предложения
- Второй слой принимает в качестве матриц $K$ и $V$ использует выход энкодера.

Декодер выполняет задачу языкового моделирования и предсказывает вероятность следующего слова. Следовательно, при обработке целевой последовательности он не может "заглядывать" вперед и анализировать контекст, который еще не был сгенерирован. Поэтому используется маскирование всех позиций после текущей, их векторы заполняются значениями `-inf`.

<center><img src="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L04/enc_dec_details.png" width="800"></center>

<center><em>Source: <a href="https://jalammar.github.io/illustrated-transformer/">The Illustrated Transformer</a></em></center>

Декодер также состоит из стека из $N=6$ идентичных слоев.

In [ ]:
class Decoder(nn.Module):
    "Generic N layer decoder with masking."
    def __init__(self, layer, N):
        super(Decoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)

    def forward(self, x, memory, src_mask, tgt_mask):
        for layer in self.layers:
            x = layer(x, memory, src_mask, tgt_mask)
        return self.norm(x)

В дополнение к двум подслоям на каждом слое энкодера, декодер содержит третий подслой, который применяет механизм множественного внимания к выходным данным энкодера. Как и в случае с энкодером, мы используем остаточные соединения вокруг каждого из подслоев с последующей нормализации слоя.

In [ ]:
class DecoderLayer(nn.Module):
    "Decoder is made of self-attn, src-attn, and feed forward (defined below)"
    def __init__(self, size, self_attn, src_attn, feed_forward, dropout):
        super(DecoderLayer, self).__init__()
        self.size = size
        self.self_attn = self_attn
        self.src_attn = src_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 3)

    def forward(self, x, memory, src_mask, tgt_mask):
        "Follow Figure 1 (right) for connections."
        m = memory
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, tgt_mask))
        x = self.sublayer[1](x, lambda x: self.src_attn(x, m, m, src_mask))
        return self.sublayer[2](x, self.feed_forward)

Мы также модифицируем подслой внутреннего внимания в стеке декодера, добавляя маскирование. Это необходимо, чтобы не учитывать токены в следующих позициях при анализе текущей и предыдущей позиций. Маскирование гарантирует, что предсказания для позиции $i$ могут зависеть только от выходных данных на позициях, меньших, чем $i$.

In [ ]:
def subsequent_mask(size):
    "Mask out subsequent positions"
    attn_shape = (1, size, size)
    subsequent_mask = np.triu(np.ones(attn_shape), k=1).astype('uint8')
    return torch.from_numpy(subsequent_mask) == 0

Маска внимания показывает позицию (строка), на которую разрешено смотреть каждому слову (столбец). Слова блокируются для того, чтобы во время обучения можно было обратить внимание на будущие слова.

In [ ]:
plt.figure(figsize=(5,5))
plt.imshow(subsequent_mask(20)[0])
None

### Трансформер

Определим функцию от гиперпараметров для построения полной модели.

<center><img src="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L04/transformer_full.png" width="1000"></center>

<center><em>Source: <a href="https://lena-voita.github.io/nlp_course/seq2seq_and_attention.html">Sequence to Sequence and Attention</a></em></center>

In [ ]:
class EncoderDecoder(nn.Module):
    """
    A standard Encoder-Decoder architecture.
    Base for this and many other models.
    """
    def __init__(self, encoder, decoder, src_embed, tgt_embed, generator):
        super(EncoderDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.generator = generator

    def forward(self, src, tgt, src_mask, tgt_mask):
        "Take in and process masked src and target sequences."
        return self.decode(self.encode(src, src_mask), src_mask,
                            tgt, tgt_mask)

    def encode(self, src, src_mask):
        return self.encoder(self.src_embed(src), src_mask)

    def decode(self, memory, src_mask, tgt, tgt_mask):
        return self.decoder(self.tgt_embed(tgt), memory, src_mask, tgt_mask)

In [ ]:
class Generator(nn.Module):
    "Define standard linear + softmax generation step."
    def __init__(self, d_model, vocab):
        super(Generator, self).__init__()
        self.proj = nn.Linear(d_model, vocab)

    def forward(self, x):
        return F.log_softmax(self.proj(x), dim=-1)

In [ ]:
def make_model(src_vocab, tgt_vocab, N=6, d_model=512, d_ff=2048, h=8, dropout=0.1):
    "Construct a model from hyperparameters"
    c = copy.deepcopy  # use it for attn, ffn, and position in the model layers
    attn = MultiHeadedAttention(h, d_model)
    ff = PositionwiseFeedForward(d_model, d_ff, dropout)
    position = PositionalEncoding(d_model, dropout)
    # insert correct arguments into the EncoderDecoder constructor.
    model = EncoderDecoder(
        Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout), N),
        Decoder(DecoderLayer(d_model, c(attn), c(attn), c(ff), dropout), N),
        nn.Sequential(Embeddings(d_model, src_vocab), c(position)),
        nn.Sequential(Embeddings(d_model, tgt_vocab), c(position)),
        Generator(d_model, tgt_vocab)
    )

    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform(p)
    return model

In [ ]:
# Small example model.
tmp_model = make_model(10, 30, 2)
assert sum(p.numel() for p in tmp_model.parameters()) == 14750750

In [ ]:
sum(p.numel() for p in tmp_model.encoder.parameters()) + sum(p.numel() for p in tmp_model.decoder.parameters()) + sum(p.numel() for p in tmp_model.tgt_embed.parameters()) + sum(p.numel() for p in tmp_model.src_embed.parameters()) + sum(p.numel() for p in tmp_model.generator.parameters())

## Обучение модели

В этом разделе описывается режим обучения для нашей модели.

### Деление на батчи и маскирование

Вначале определим формат батча, который содержит исходное и целевое предложения для обучения, а также создание масок.

In [ ]:
class Batch:
    "Object for holding a batch of data with mask during training."
    def __init__(self, src, trg=None, pad=0):
        self.src = src
        self.src_mask = (src != pad).unsqueeze(-2)
        if trg is not None:
            self.trg = trg[:, :-1]
            self.trg_y = trg[:, 1:]
            self.trg_mask = \
                self.make_std_mask(self.trg, pad)
            self.ntokens = (self.trg_y != pad).data.sum()

    @staticmethod
    def make_std_mask(tgt, pad):
        "Create a mask to hide padding and future words."
        tgt_mask = (tgt != pad).unsqueeze(-2)
        tgt_mask = tgt_mask & Variable(
            subsequent_mask(tgt.size(-1)).type_as(tgt_mask.data))
        return tgt_mask

Создадим батчи с помощью текстовой функции torch, которая гарантирует, что размер нашего батча после паддинга до максимального размера батча не превысит порогового значения.

In [ ]:
global max_src_in_batch, max_tgt_in_batch
def batch_size_fn(new, count, sofar):
    "Keep augmenting batch and calculate total number of tokens + padding."
    global max_src_in_batch, max_tgt_in_batch
    if count == 1:
        max_src_in_batch = 0
        max_tgt_in_batch = 0
    max_src_in_batch = max(max_src_in_batch,  len(new.src))
    max_tgt_in_batch = max(max_tgt_in_batch,  len(new.trg) + 2)
    src_elements = count * max_src_in_batch
    tgt_elements = count * max_tgt_in_batch
    return max(src_elements, tgt_elements)

### Цикл обучения

Затем создадим функцию для обучения, подсчета значения функции потерь и обновления параметров.

In [ ]:
import time

def run_epoch(data_iter, model, loss_compute):
    "Standard Training and Logging Function"
    start = time.time()
    total_tokens = 0
    total_loss = 0
    tokens = 0
    for i, batch in enumerate(data_iter):
        out = model.forward(batch.src, batch.trg,
                            batch.src_mask, batch.trg_mask)
        loss = loss_compute(out, batch.trg_y, batch.ntokens)
        total_loss += loss
        total_tokens += batch.ntokens
        tokens += batch.ntokens
        if i % 50 == 1:
            elapsed = time.time() - start
            print("Epoch Step: %d Loss: %f Tokens per Sec: %f" %
                    (i, loss / batch.ntokens, tokens / elapsed))
            start = time.time()
            tokens = 0
    return total_loss / total_tokens

### Оптимизатор

Будем использовать оптимизатор `Adam` [🛠️[doc]](https://pytorch.org/docs/stable/generated/torch.optim.Adam.html) с параметрами $\beta_1=0.9$, $\beta_2=0.98$ и $\epsilon=10^{-9}$.  Скорость обучения будет варьироваться в соответствии с формулой:
$$
lrate = d_{\text{model}}^{-0.5} \cdot
  \min({step\_num}^{-0.5},
    {step\_num} \cdot {warmup\_steps}^{-1.5})
$$

Это соответствует линейному увеличению скорости обучения на первых этапах $warmup\_steps$ и последующему ее уменьшению пропорционально обратному квадратному корню из номера шага. Будем использовать значение $warmup\_steps=4000$.

In [ ]:
class NoamOpt:
    "Optim wrapper that implements rate."
    def __init__(self, model_size, factor, warmup, optimizer):
        self.optimizer = optimizer
        self._step = 0
        self.warmup = warmup
        self.factor = factor
        self.model_size = model_size
        self._rate = 0

    def step(self):
        "Update parameters and rate"
        self._step += 1
        rate = self.rate()
        for p in self.optimizer.param_groups:
            p['lr'] = rate
        self._rate = rate
        self.optimizer.step()

    def rate(self, step = None):
        "Implement `lrate` above"
        if step is None:
            step = self._step
        return self.factor * \
            (self.model_size ** (-0.5) *
            min(step ** (-0.5), step * self.warmup ** (-1.5)))

def get_std_opt(model):
    return NoamOpt(model.src_embed[0].d_model, 2, 4000,
            torch.optim.Adam(model.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9))

Пример графиков для различных размеров модели и гиперпараметров оптимизации.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
# Three settings of the lrate hyperparameters.
opts = [NoamOpt(512, 1, 4000, None),
        NoamOpt(512, 1, 8000, None),
        NoamOpt(256, 1, 4000, None)]
plt.plot(np.arange(1, 20000), [[opt.rate(i) for opt in opts] for i in range(1, 20000)])
plt.legend(["512:4000", "512:8000", "256:4000"])
None

### Регуляризация

Сглаживание меток — это метод, используемый для регуляризации процесса обучения нейронной сети. Он помогает избежать переобучения за счет уменьшения зависимости модели от правильных меток во время процесса обучения. Это работает за счет повышения уверенности модели в «неправильных» ярлыках.

Во время обучения будем использовать сглаживание значений меток с параметром $\varepsilon_{ls}=0.1$. Вместо one-hot распределения по одной целевой группе мы создаем распределение, в котором наибольшую вероятность имеет правильное слово, а остальные значения вероятности распределяются по всему словарю.

\begin{align*}
q_i =
\left\{
    \begin {aligned}
         & 1 - \varepsilon \quad & \text{if } i=y, \\
         & \varepsilon/(K-1) \quad & \text{otherwise}                  
    \end{aligned}
\right.
\end{align*}

<center><img src="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L04/label_smoothing.png" width="700"></center>

<center><em>Source: <a href="https://arxiv.org/abs/2011.12562">Delving Deep into Label Smoothing</a></em></center>

In [ ]:
class LabelSmoothing(nn.Module):
    def __init__(self, size, padding_idx, smoothing=0.0):
        super(LabelSmoothing, self).__init__()
        self.criterion = nn.KLDivLoss(size_average=False)
        self.padding_idx = padding_idx
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.size = size
        self.true_dist = None

    def forward(self, x, target):
        assert x.size(1) == self.size
        true_dist = x.data.clone()
        true_dist.fill_(self.smoothing / (self.size - 2))
        true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        true_dist[:, self.padding_idx] = 0
        mask = torch.nonzero(target.data == self.padding_idx)
        if mask.dim() > 0:
            true_dist.index_fill_(0, mask.squeeze(), 0.0)
        self.true_dist = true_dist
        return self.criterion(x, Variable(true_dist, requires_grad=False))